# CS565-DS522 IoT Data Science Mini Project for K-EmoPhone dataset
*This material is a joint work of TAs from IC Lab at KAIST, including Panyu Zhang, Soowon Kang, and Woohyeok Choi. This work is licensed under CC BY-SA 4.0.*

## Instruction
In this mini-project, we will build a model to predict users' self-reported stress using extracted features from K-EmoPhone dataset. This material mainly refers to the public [repository](https://github.com/SteinPanyu/IndependentReproducibility) conducting indepedent reproducibility experiments on K-EmoPhone dataset. In order to save time, we provide the extracted features from the raw data instead of starting from scratch. Besides, traditional machine learning model is used considering limited number of labels and multimodality issue in the in-the-wild K-EmoPhone dataset.



## Guidance

1. Before running the code, please first download the extracted features from the following [link](https://drive.google.com/file/d/1HcyFvzWEzO21osyP5E8VpVmHROX1ew7q/view?usp=sharing).

2. Please change your runtime type to T4-GPU or other runtime types with GPU available since later we may use GPU for
xgboost execution

Install latest version of xgboost > 2.0.0

In [7]:
!pip install xgboost

In [8]:
import pytz
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import cloudpickle
from datetime import datetime
from contextlib import contextmanager
import warnings
import time
from typing import Optional
from contextlib import contextmanager

DEFAULT_TZ = pytz.FixedOffset(540)  # GMT+09:00; Asia/Seoul

RANDOM_STATE =42


def log(msg: any):
    print('[{}] {}'.format(datetime.now().strftime('%y-%m-%d %H:%M:%S'), msg))

## 1.Preparation

### 1.1. Mount to Your Google Drive

In [5]:
# not relevant for local execution
'''
from google.colab import drive

drive.mount('/content/drive')
'''

"\nfrom google.colab import drive\n\ndrive.mount('/content/drive')\n"

### 1.2. Load Extracted Features

In [9]:
import pickle
import numpy as np

#PATH = '/content/drive/MyDrive/IoT_Data_Science/Project/Datasets/features_stress_fixed_K-EmoPhone.pkl'
PATH = './Datasets/features_stress_fixed_K-EmoPhone.pkl'

X, y, groups, t, datetimes = pickle.load(open(PATH, mode='rb'))

X is the extracted features and the feature extraction process refers to the public [repository](https://github.com/SteinPanyu/IndependentReproducibility) and the immediate past time window is set as 15 minutes. y is the array of labels while groups is the user ids.

Please note that here y is binarized using theoretical threshold (if ESM stress > 0, binarize as 1, else 0, ESM label scale [-3, 3])

Since features are already extracted, we do not need to work on preprocessing and feature extraction again.

## 2.Feature Preparation


There exist multiple types of features. Please try different combinations of features to see if there is any model performance improvement.

In [10]:

#The following code is designed for reordering the data
#################################################
# Create a DataFrame with user_id and datetime

df = pd.DataFrame({'user_id': groups, 'datetime': datetimes, 'label': y})

# df_merged = pd.merge(df, X, left_index=True, right_index=True)
df_merged = pd.merge(df, X, left_index=True, right_index=True)

# Sort the DataFrame by datetime
df_merged = df_merged.sort_values(by=['user_id', 'datetime'])

# Update groups and datetimes
groups = df_merged['user_id'].to_numpy()
datetimes = df_merged['datetime'].to_numpy()
y = df_merged['label'].to_numpy()
X = df_merged.drop(columns=['user_id', 'datetime', 'label'])



#Divide the features into different categories
feat_current = X.loc[:,[('#VAL' in str(x)) or ('ESM#LastLabel' in str(x)) for x in X.keys()]]
feat_dsc = X.loc[:,[('#DSC' in str(x))  for x in X.keys()]]
feat_yesterday = X.loc[:,[('Yesterday' in str(x))  for x in X.keys()]]
feat_today = X.loc[:,[('Today' in str(x))  for x in X.keys()]]

feat_ImmediatePast = X.loc[:,[('ImmediatePast_15' in str(x))  for x in X.keys()]]

#################################################################################
#Below are the available features
#Divide the time window features into sensor/ESM self-report features
feat_current_sensor = X.loc[:,[('#VAL' in str(x))  for x in X.keys()]] #Current sensor features (value right before label)
feat_current_ESM = X.loc[:,[('ESM#LastLabel' in str(x)) for x in X.keys()]] #Current ESM features (value right before label)
feat_ImmediatePast_sensor = feat_ImmediatePast.loc[:,[('ESM' not in str(x)) for x in feat_ImmediatePast.keys()]] #Immediate past sensor features (in past 15 minutes before label)
feat_ImmediatePast_ESM = feat_ImmediatePast.loc[:,[('ESM'  in str(x)) for x in feat_ImmediatePast.keys()]]  #Immediate past ESM features
feat_today_sensor = feat_today.loc[:,[('ESM' not in str(x))  for x in feat_today.keys()]] #Today epoch sensor features
feat_today_ESM = feat_today.loc[:,[('ESM'  in str(x)) for x in feat_today.keys()]] #Today epoch ESM features
feat_yesterday_sensor = feat_yesterday.loc[:,[('ESM' not in str(x)) for x in feat_yesterday.keys()]] #Yesterday sensor features
feat_yesterday_ESM = feat_yesterday.loc[:,[('ESM'  in str(x)) for x in feat_yesterday.keys()]] #Yesterday ESM features

feat_sleep = X.loc[:,[('Sleep' in str(x))  for x in X.keys()]]
feat_time = X.loc[:,[('Time' in str(x))  for x in X.keys()]]
feat_pif = X.loc[:,[('PIF' in str(x))  for x in X.keys()]]
################################################################################

#Prepare the final feature set
feat_baseline = pd.concat([ feat_time,feat_dsc,feat_current_sensor, feat_ImmediatePast_sensor],axis=1)

feat_final = pd.concat([feat_baseline  ],axis=1)


################################################################################
X = feat_final
cats = X.columns[X.dtypes == bool]

In [11]:
feat_current_ESM

,ESM#LastLabel
0,0.0
1,1.0
2,1.0
3,0.0
4,0.0
...,...
2614,0.0
2615,0.0
2616,0.0
2617,1.0


## 3.Model Training & Evaluation


Here is the revised XGBoost Classifier. We will use random eval_size percent of training set data as evaluation set for early stoppping.

In [12]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, DMatrix
from sklearn.base import BaseEstimator
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from typing import Union

#Function for revised xgboost classifier
class EvXGBClassifier(BaseEstimator):
    """
    Enhanced XGBClassifier with built-in validation set approach for early stopping.
    """
    def __init__(
        self,
        eval_size=None,
        eval_metric='logloss',
        early_stopping_rounds=10,
        random_state=None,
        **kwargs
        ):
        """
        Initializes the custom XGBoost Classifier.

        Args:
            eval_size (float): The proportion of the dataset to include in the evaluation split.
            eval_metric (str): The evaluation metric used for model training.
            early_stopping_rounds (int): The number of rounds to stop training if hold-out metric doesn't improve.
            random_state (int): Seed for the random number generator for reproducibility.
            **kwargs: Additional arguments to be passed to the underlying XGBClassifier.
        """
        self.random_state = random_state
        self.eval_size = eval_size
        self.eval_metric = eval_metric
        self.early_stopping_rounds = early_stopping_rounds
        # Initialize the XGBClassifier with specified arguments and GPU acceleration.
        self.model = XGBClassifier(
            random_state=self.random_state,
            eval_metric=self.eval_metric,
            early_stopping_rounds=self.early_stopping_rounds,
            tree_method = "hist", device = "cuda", #Use gpu for acceleration
            **kwargs
        )

    @property
    def feature_importances_(self):
        """ Returns the feature importances from the fitted model. """
        return self.model.feature_importances_

    @property
    def feature_names_in_(self):
        """ Returns the feature names from the input dataset used for fitting. """
        return self.model.feature_names_in_

    def fit(self, X: Union[pd.DataFrame, np.ndarray], y: np.ndarray):
        """
        Fit the XGBoost model with optional early stopping using a validation set.

        Args:
            X (Union[pd.DataFrame, np.ndarray]): Training features.
            y (np.ndarray): Target values.
        """
        if self.eval_size:
            # Split data for early stopping evaluation if eval_size is specified.
            X_train_sub, X_val, y_train_sub, y_val = train_test_split(
                X, y, test_size=self.eval_size, random_state=self.random_state)
            # Fit the model with early stopping.
            self.model.fit(
                X_train_sub, y_train_sub,
                eval_set=[(X_val, y_val)],
                verbose=False
            )
        else:
            # Fit the model without early stopping.
            self.model.fit(X, y, verbose=False)

        # Store the best iteration number for predictions.
        self.best_iteration_ = self.model.get_booster().best_iteration
        return self

    def predict(self, X: pd.DataFrame):
        """
        Predict the classes for the given features.

        Args:
            X (pd.DataFrame): Input features.
        """
        return self.model.predict(X, iteration_range=(0, self.best_iteration_ + 1))

    def predict_proba(self, X: pd.DataFrame):
        """
        Predict the class probabilities for the given features.

        Args:
            X (pd.DataFrame): Input features.
        """
        return self.model.predict_proba(X, iteration_range=(0, self.best_iteration_ + 1))

The following is defined functions for model training and model evaluation (cross-validation).

In [ ]:
import os
import pandas as pd
import numpy as np
import time
import traceback
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, LeaveOneGroupOut, StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
from dataclasses import dataclass

@dataclass
class FoldResult:
    name: str
    metrics: dict
    duration: float

def log(message: str):
    print(message)  # Simple logging to stdout or enhance as needed

def train_fold(dir_result: str, fold_name: str, X_train, y_train, X_test, y_test, C_cat, C_num, estimator, normalize, select, oversample, random_state):
    """
    Function to train and evaluate the model for a single fold.
    Args:
        dir_result (str): Directory to store results.
        fold_name (str): Name of the fold for identification.
        X_train, y_train (DataFrame, Series): Training data.
        X_test, y_test (DataFrame, Series): Testing data.
        C_cat, C_num (array): Lists of categorical and numeric feature names.
        estimator (estimator instance): The model to be trained.
        normalize (bool): Flag to apply normalization.
        select (SelectFromModel instance): Feature selection method.
        oversample (bool): Flag to apply oversampling.
        random_state (int): Random state for reproducibility.
    Returns:
        FoldResult: Object containing metrics and duration of the training.
    """
    try:
        start_time = time.time()
        if normalize:
            X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
            X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values
            # Standard scaler only applied to numeric data
            scaler = StandardScaler().fit(X_train_N)
            X_train_N = scaler.transform(X_train_N)
            X_test_N = scaler.transform(X_test_N)

            X_train = pd.DataFrame(
                np.concatenate((X_train_C, X_train_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
            X_test = pd.DataFrame(
                np.concatenate((X_test_C, X_test_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )

        #Applying the LASSO feature selection method
        if select:

            if isinstance(select, SelectFromModel):
                select = [select]

            for i, s in enumerate(select):
                C = np.asarray(X_train.columns)
                M = s.fit(X=X_train.values, y=y_train).get_support()
                C_sel = C[M]
                C_cat = C_cat[np.isin(C_cat, C_sel)]
                C_num = C_num[np.isin(C_num, C_sel)]

                X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
                X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values


                X_train = pd.DataFrame(
                    np.concatenate((X_train_C, X_train_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )
                X_test = pd.DataFrame(
                    np.concatenate((X_test_C, X_test_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )

        if oversample:
            # TODO: CHANGE TO ADASYN
            #If there is any categorical data, apply SMOTE-NC, otherwise just SMOTE
            if len(C_cat) > 0:
                sampler = SMOTENC(categorical_features=[X_train.columns.get_loc(c) for c in C_cat], random_state=random_state)
            else:
                sampler = SMOTE(random_state=random_state)
            X_train, y_train = sampler.fit_resample(X_train, y_train)

        estimator = clone(estimator).fit(X_train, y_train)
        y_pred = estimator.predict_proba(X_test)[:, 1]
        #Deafult average method for roc_auc_score is macro
        auc_score = roc_auc_score(y_test, y_pred, average=None)

        result = FoldResult(
            name=fold_name,
            metrics={'AUC': auc_score},
            duration=time.time() - start_time
        )
        log(f'Training completed for {fold_name} with AUC: {auc_score}')
        return result

    except Exception as e:
        log(f'Error in {fold_name}: {traceback.format_exc()}')
        return None

def perform_cross_validation(X, y, groups, estimator, normalize=False, select=None, oversample=False, random_state=None):
    """
    Function to perform cross-validation using StratifiedGroupKFold.
    Args:
        X, y (DataFrame, Series): The entire dataset.
        groups (array): Array indicating the group for each instance in X.
        estimator (estimator instance): The model to be trained.
        normalize, select, oversample (bool): Preprocessing options.
        random_state (int): Seed for reproducibility.
    Returns:
        list: A list containing FoldResult for each fold.
    """
    futures = []
    # Group-k cross validation
    splitter = StratifiedGroupKFold(n_splits=5, shuffle =True, random_state = 42)
    # Loop over all the LOSO splits
    for idx, (train_idx, test_idx) in enumerate(splitter.split(X, y, groups)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        C_cat = np.asarray(sorted(cats))
        C_num = np.asarray(sorted(X.columns[~X.columns.isin(C_cat)]))

        job = train_fold('path_to_results', f'Fold_{idx}', X_train, y_train, X_test, y_test, C_cat, C_num, estimator, normalize, select, oversample, random_state)
        futures.append(job)

    return futures

Here, we define the feature selection method and classifier and execute the code. AUC-ROC is calculated as mean of macro AUC-ROC for all folds/users.

In [14]:
#Featur Selection, you may want to change the feature selection methods
SELECT_LASSO = SelectFromModel(
        estimator=LogisticRegression(
        penalty='l1'
        ,solver='liblinear'
        , C=1, random_state=RANDOM_STATE, max_iter=4000
    ),
    # This threshold may impact the model performance as well
    threshold = 0.005
)
#Classifier
#There could exist more parameters. Please search in your defined parameter
#space for model performance improvement
estimator = EvXGBClassifier(
    random_state=RANDOM_STATE,
    eval_metric='logloss',
    eval_size=0.2,
    early_stopping_rounds=10,
    objective='binary:logistic', #Prediction instead of regression
    verbosity=0,
    learning_rate=0.01,
)

#Perform cross validation including model training and evaluation
results = perform_cross_validation(X, y, groups, estimator, normalize=True, select=[SELECT_LASSO], oversample=True, random_state=42)
auc_values = [results[i].metrics['AUC'] for i in range(len(results))]
mean_auc = np.mean(auc_values)
print(mean_auc)

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:39] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:39] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()


Training completed for Fold_0 with AUC: 0.610757167360941


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:44] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:44] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()


Training completed for Fold_1 with AUC: 0.5570281366984664


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:48] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:48] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()


Training completed for Fold_2 with AUC: 0.5699012255350283


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:53] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:53] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()


Training completed for Fold_3 with AUC: 0.506083202511774


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:57] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:57] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()


Training completed for Fold_4 with AUC: 0.5600147275405007
0.5607568919293421


# Assignment

## Assignment 1. Improve the model performance using different types of feature combinations. (20pts)

 Hint: Currently we are only using feat_baseline. You may want to try other feature combinations.

In [28]:
#######You may need to go back to the feature preparation code and check#########
feat_recent_ESM = pd.concat([feat_current_ESM,feat_ImmediatePast_ESM ], axis=1)

feat_final = pd.concat([feat_recent_ESM, feat_time],axis=1)

X = feat_final
cats = X.columns[X.dtypes == bool]


In [18]:

results = perform_cross_validation(X, y, groups, estimator, normalize=True, select=[SELECT_LASSO], oversample=True, random_state=42)
auc_values = [results[i].metrics['AUC'] for i in range(len(results))]
mean_auc = np.mean(auc_values)
print(mean_auc)

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.

Training completed for Fold_0 with AUC: 0.6745160499877482
Training completed for Fold_1 with AUC: 0.5925673227871029


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.

Training completed for Fold_2 with AUC: 0.5935339308578745
Training completed for Fold_3 with AUC: 0.6148589930477686
Training completed for Fold_4 with AUC: 0.6126336684382403
0.6176219930237469


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:14:22] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()


## Assignment 2. Please try different feature selection methods (20pts)

Hint: Currently, we are using LASSO filter for feature selection. Please consider using embedded method as well(same model for both feature selection and model training). Besides, the threshold for LASSO filter may also affect the performance. **Sepcifically, there is a method called 'mean' which is using mean of feature importances of all features as threshold.** Please try both different feature selection methods and different thresholds for filtering features to improve model performance.

In [22]:
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel

xgb_selector = SelectFromModel(
    estimator=XGBClassifier(random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss'),
    threshold='mean' # Or a different threshold
)

results = perform_cross_validation(X, y, groups, estimator, normalize=True, select=[xgb_selector], oversample=True, random_state=42)
auc_values = [results[i].metrics['AUC'] for i in range(len(results))]
mean_auc = np.mean(auc_values)
print(mean_auc)

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:44:10] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:44:11] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:44:11] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:44:11] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Training completed for Fold_0 with AUC: 0.6757534917912276
Training completed for Fold_1 with AUC: 0.6017932616833715


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:44:11] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:44:11] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:44:11] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:44:11] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CP

Training completed for Fold_2 with AUC: 0.6393451618803732
Training completed for Fold_3 with AUC: 0.6202273491814307
Training completed for Fold_4 with AUC: 0.6385029134917077
0.6351244356056223


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:44:11] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:44:11] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:44:11] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()


## Assignment 3. Please try using hyperopt for model hyperparameter tuning (20 pts)

Hint: Please be aware that for revised xgboost classifier EvXGBClassifier, there exist other parameters other than default XGBClassifier parameters such as eval_size.

For hyperparameter tuning, we will use 20% of training set as validation set to avoid data leakage.

If it is too timeconsuming to run the code in colab, please run the code locally and consider using [ray tune](https://docs.ray.io/en/latest/tune/index.html) if needed.

In [23]:
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, hp, fmin, tpe
from sklearn.model_selection import StratifiedGroupKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.feature_selection import SelectFromModel

# define your outer CV
OUTER_CV = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

def objective(params):
    val_scores = []

    # outer loop: split into train_full / test (we will only use train_full for tuning)
    for train_full_idx, _ in OUTER_CV.split(X, y, groups):
        X_train_full = X.iloc[train_full_idx]
        y_train_full = y[train_full_idx]

        # split 20% of the *training fold* into a validation set
        X_train, X_val, y_train, y_val = train_test_split(
            X_train_full, y_train_full,
            test_size=0.20,
            stratify=y_train_full,
            random_state=42
        )

        # 1) Normalize
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled   = scaler.transform(X_val)

        # 2) (Optional) Oversample on *training only*
        if np.any(X_train_scaled[:, -1] < 1):
            smote = SMOTENC(
                categorical_features=[X_train_scaled.shape[1]-1],
                random_state=int(params['random_state'])
            )
        else:
            smote = SMOTE(random_state=int(params['random_state']))
        X_train_os, y_train_os = smote.fit_resample(X_train_scaled, y_train)

        # 3) Feature selection on *training only*
        selector = SelectFromModel(
            LogisticRegression(penalty='l1', solver='liblinear',
                               random_state=int(params['random_state'])),
            threshold='mean'
        )
        X_train_sel = selector.fit_transform(X_train_os, y_train_os)
        X_val_sel   = selector.transform(X_val_scaled)

        # 4) Train & score on *validation only*
        clf = LogisticRegression(
            random_state=int(params['random_state']),
            max_iter=1000
        )
        clf.fit(X_train_sel, y_train_os)
        y_val_prob = clf.predict_proba(X_val_sel)[:, 1]
        val_scores.append(roc_auc_score(y_val, y_val_prob))

    # Hyperopt minimizes “loss”, so negate AUC
    return {'loss': -np.mean(val_scores), 'status': STATUS_OK}


# define your search space (fill in any missing parameters e.g. max_depth)
space = {
    'max_depth':          hp.quniform('max_depth', 3, 10, 1),  # Integer between 3 and 10
    'min_child_weight':   hp.quniform('min_child_weight', 1, 10, 1), # Integer between 1 and 10
    'subsample':          hp.uniform('subsample', 0.6, 1.0),  # Float between 0.6 and 1.0
    'colsample_bytree':   hp.uniform('colsample_bytree', 0.6, 1.0), # Float between 0.6 and 1.0
    'gamma':              hp.uniform('gamma', 0, 0.5),      # Float between 0 and 0.5
    'learning_rate':      hp.loguniform('learning_rate', -5, 0), # Float on a log scale (0.0067 to 1)
    'n_estimators':       hp.quniform('n_estimators', 100, 1000, 50), # Integer between 100 and 1000, steps of 50
    'reg_lambda':         hp.uniform('reg_lambda', 0, 1),     # Float between 0 and 1 (L2 regularization)
    'reg_alpha':          hp.uniform('reg_alpha', 0, 0.5),    # Float between 0 and 0.5 (L1 regularization)
    'random_state':       42 # Keeping random_state fixed
}

# run hyperopt
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

print("Best hyperparameters:", best)

100%|██████████| 100/100 [00:47<00:00,  2.10trial/s, best loss: -0.6189693231281869]
Best hyperparameters: {'colsample_bytree': 0.9012966810519976, 'gamma': 0.22247273381633864, 'learning_rate': 0.028542060596548048, 'max_depth': 7.0, 'min_child_weight': 8.0, 'n_estimators': 800.0, 'reg_alpha': 0.2608104132350774, 'reg_lambda': 0.2620754838361904, 'subsample': 0.8360036468008175}


In [36]:

#Classifier
estimator = EvXGBClassifier(
    colsample_bytree=best['colsample_bytree'],
    gamma=best['gamma'],
    learning_rate=best['learning_rate'],
    max_depth=int(best['max_depth']),
    min_child_weight=int(best['min_child_weight']),
    n_estimators=int(best['n_estimators']),
    reg_alpha=best['reg_alpha'],
    reg_lambda=best['reg_lambda'],
    subsample=best['subsample'],
    random_state=RANDOM_STATE,
    eval_metric='logloss',
    eval_size=0.2,
    early_stopping_rounds=10,
    objective='binary:logistic', #Prediction instead of regression
    verbosity=0,
    use_label_encoder=False,  # Avoid warning about label encoder
)

results = perform_cross_validation(X, y, groups, estimator, normalize=True, select=[xgb_selector], oversample=True, random_state=42)
auc_values = [results[i].metrics['AUC'] for i in range(len(results))]
mean_auc = np.mean(auc_values)
print(mean_auc)

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:52:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [16:52:19] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [16:52:19] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()


Training completed for Fold_0 with AUC: 0.6757534917912276
Training completed for Fold_1 with AUC: 0.6017932616833715


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:52:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [16:52:19] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [16:52:19] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:52:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Training completed for Fold_2 with AUC: 0.6393451618803732
Training completed for Fold_3 with AUC: 0.6202273491814307
Training completed for Fold_4 with AUC: 0.6385029134917077
0.6351244356056223


/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [16:52:20] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [16:52:20] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  self.starting_round = model.num_boosted_rounds()


## Assignment 4. Please consider replacing the previous traditional machine learning model with deep learning models designed for **tabular data** to improve model performance. (20 pts)

Hint: Since features are already extracted manually, it is impossible to use end-to-end deep learning models. Instead, try replacing xgboost with deep learning models designed for **tabular data** and see if there is performance improvement.

In [34]:
from sklearn.base import BaseEstimator, ClassifierMixin
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

class SklearnTabNet(BaseEstimator, ClassifierMixin):
    def __init__(self, **kwargs):
        self.model = TabNetClassifier(**kwargs)
        self.fit_params = kwargs
        self.label_encoder = LabelEncoder()

    def fit(self, X, y):
        # Convert DataFrame to NumPy if needed
        if hasattr(X, "values"):
            X = X.values
        y = self.label_encoder.fit_transform(y)
        self.model.fit(X, y)
        return self

    def predict(self, X):
        if hasattr(X, "values"):
            X = X.values
        preds = self.model.predict(X)
        return self.label_encoder.inverse_transform(preds)

    def predict_proba(self, X):
        if hasattr(X, "values"):
            X = X.values
        return self.model.predict_proba(X)


You may need to change runtime to TPU first to use torch or other packages you may want to use.



Please compare it with your previous XGBoost model performance and think about why it is higher or lower than XGBoost.

In [37]:
import torch
#######Your code for deep learning model#########

estimator = SklearnTabNet(
    n_d=8,
    n_a=8,
    n_steps=3,
    gamma=1.3,
    cat_idxs=[X.columns.get_loc(c) for c in cats],
    cat_dims=[2] * len(cats),  # Assuming binary categorical features
    cat_emb_dim=1,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params=dict(step_size=50, gamma=0.9),
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=0
)
results = perform_cross_validation(X, y, groups, estimator, normalize=True, select=[xgb_selector], oversample=True, random_state=42)
auc_values = [results[i].metrics['AUC'] for i in range(len(results))]
mean_auc = np.mean(auc_values)
print(mean_auc)

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:53:25] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.82129 |  0:00:00s
epoch 1  | loss: 0.69413 |  0:00:00s
epoch 2  | loss: 0.68169 |  0:00:00s
epoch 3  | loss: 0.67449 |  0:00:00s
epoch 4  | loss: 0.66989 |  0:00:00s
epoch 5  | loss: 0.66785 |  0:00:00s
epoch 6  | loss: 0.66801 |  0:00:00s
epoch 7  | loss: 0.6692  |  0:00:00s
epoch 8  | loss: 0.67093 |  0:00:00s
epoch 9  | loss: 0.66066 |  0:00:00s
epoch 10 | loss: 0.66997 |  0:00:00s
epoch 11 | loss: 0.66499 |  0:00:00s
epoch 12 | loss: 0.6652  |  0:00:00s
epoch 13 | loss: 0.66063 |  0:00:00s
epoch 14 | loss: 0.66369 |  0:00:00s
epoch 15 | loss: 0.66042 |  0:00:00s
epoch 16 | loss: 0.66606 |  0:00:01s
epoch 17 | loss: 0.6549  |  0:00:01s
epoch 18 | loss: 0.66593 |  0:00:01s
epoch 19 | loss: 0.66275 |  0:00:01s
epoch 20 | loss: 0.66116 |  0:00:01s
epoch 21 | loss: 0.67011 |  0:00:01s
epoch 22 | loss: 0.66674 |  0:00:01s
epoch 23 | loss: 0.66347 |  0:00:01s
epoch 24 | loss: 0.66219 |  0:00:01s
epoch 25 | loss: 0.66776 |  0:00:01s
epoch 26 | loss: 0.66323 |  0:00:01s
e

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:53:32] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 1  | loss: 0.66669 |  0:00:00s
epoch 2  | loss: 0.66416 |  0:00:00s
epoch 3  | loss: 0.65561 |  0:00:00s
epoch 4  | loss: 0.65932 |  0:00:00s
epoch 5  | loss: 0.66116 |  0:00:00s
epoch 6  | loss: 0.65419 |  0:00:00s
epoch 7  | loss: 0.65517 |  0:00:00s
epoch 8  | loss: 0.66334 |  0:00:00s
epoch 9  | loss: 0.65327 |  0:00:00s
epoch 10 | loss: 0.65971 |  0:00:00s
epoch 11 | loss: 0.659   |  0:00:00s
epoch 12 | loss: 0.66051 |  0:00:00s
epoch 13 | loss: 0.65711 |  0:00:00s
epoch 14 | loss: 0.6592  |  0:00:00s
epoch 15 | loss: 0.65609 |  0:00:00s
epoch 16 | loss: 0.65753 |  0:00:00s
epoch 17 | loss: 0.66103 |  0:00:01s
epoch 18 | loss: 0.66029 |  0:00:01s
epoch 19 | loss: 0.65484 |  0:00:01s
epoch 20 | loss: 0.65865 |  0:00:01s
epoch 21 | loss: 0.65416 |  0:00:01s
epoch 22 | loss: 0.65564 |  0:00:01s
epoch 23 | loss: 0.65826 |  0:00:01s
epoch 24 | loss: 0.65388 |  0:00:01s
epoch 25 | loss: 0.65951 |  0:00:01s
epoch 26 | loss: 0.65773 |  0:00:01s
epoch 27 | loss: 0.6569  |  0:00:01s
e

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:53:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.74116 |  0:00:00s
epoch 1  | loss: 0.70513 |  0:00:00s
epoch 2  | loss: 0.69432 |  0:00:00s
epoch 3  | loss: 0.67477 |  0:00:00s
epoch 4  | loss: 0.67408 |  0:00:00s
epoch 5  | loss: 0.66448 |  0:00:00s
epoch 6  | loss: 0.66114 |  0:00:00s
epoch 7  | loss: 0.67036 |  0:00:00s
epoch 8  | loss: 0.66511 |  0:00:00s
epoch 9  | loss: 0.66068 |  0:00:00s
epoch 10 | loss: 0.66214 |  0:00:00s
epoch 11 | loss: 0.6624  |  0:00:00s
epoch 12 | loss: 0.66165 |  0:00:00s
epoch 13 | loss: 0.66099 |  0:00:00s
epoch 14 | loss: 0.66227 |  0:00:00s
epoch 15 | loss: 0.66577 |  0:00:00s
epoch 16 | loss: 0.6587  |  0:00:00s
epoch 17 | loss: 0.66433 |  0:00:00s
epoch 18 | loss: 0.6621  |  0:00:00s
epoch 19 | loss: 0.6587  |  0:00:00s
epoch 20 | loss: 0.65557 |  0:00:00s
epoch 21 | loss: 0.65922 |  0:00:00s
epoch 22 | loss: 0.66284 |  0:00:00s
epoch 23 | loss: 0.65952 |  0:00:00s
epoch 24 | loss: 0.66066 |  0:00:00s
epoch 25 | loss: 0.65914 |  0:00:01s
epoch 26 | loss: 0.65793 |  0:00:01s
e

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:53:40] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.82813 |  0:00:00s
epoch 1  | loss: 0.67173 |  0:00:00s
epoch 2  | loss: 0.65725 |  0:00:00s
epoch 3  | loss: 0.65813 |  0:00:00s
epoch 4  | loss: 0.65162 |  0:00:00s
epoch 5  | loss: 0.65938 |  0:00:00s
epoch 6  | loss: 0.65699 |  0:00:00s
epoch 7  | loss: 0.65609 |  0:00:00s
epoch 8  | loss: 0.65957 |  0:00:00s
epoch 9  | loss: 0.65455 |  0:00:00s
epoch 10 | loss: 0.65117 |  0:00:00s
epoch 11 | loss: 0.64928 |  0:00:00s
epoch 12 | loss: 0.6484  |  0:00:00s
epoch 13 | loss: 0.65202 |  0:00:00s
epoch 14 | loss: 0.65083 |  0:00:00s
epoch 15 | loss: 0.65236 |  0:00:00s
epoch 16 | loss: 0.65407 |  0:00:00s
epoch 17 | loss: 0.65722 |  0:00:00s
epoch 18 | loss: 0.65453 |  0:00:00s
epoch 19 | loss: 0.65006 |  0:00:00s
epoch 20 | loss: 0.65286 |  0:00:00s
epoch 21 | loss: 0.64635 |  0:00:00s
epoch 22 | loss: 0.65601 |  0:00:00s
epoch 23 | loss: 0.65498 |  0:00:01s
epoch 24 | loss: 0.65045 |  0:00:01s
epoch 25 | loss: 0.64957 |  0:00:01s
epoch 26 | loss: 0.64914 |  0:00:01s
e

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:53:45] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.16024 |  0:00:00s
epoch 1  | loss: 0.71131 |  0:00:00s
epoch 2  | loss: 0.70045 |  0:00:00s
epoch 3  | loss: 0.69166 |  0:00:00s
epoch 4  | loss: 0.67844 |  0:00:00s
epoch 5  | loss: 0.67201 |  0:00:00s
epoch 6  | loss: 0.66742 |  0:00:00s
epoch 7  | loss: 0.65627 |  0:00:00s
epoch 8  | loss: 0.65846 |  0:00:00s
epoch 9  | loss: 0.66    |  0:00:00s
epoch 10 | loss: 0.66268 |  0:00:00s
epoch 11 | loss: 0.65486 |  0:00:00s
epoch 12 | loss: 0.66073 |  0:00:00s
epoch 13 | loss: 0.66184 |  0:00:00s
epoch 14 | loss: 0.66056 |  0:00:00s
epoch 15 | loss: 0.65274 |  0:00:00s
epoch 16 | loss: 0.66015 |  0:00:00s
epoch 17 | loss: 0.66601 |  0:00:00s
epoch 18 | loss: 0.66265 |  0:00:00s
epoch 19 | loss: 0.65902 |  0:00:00s
epoch 20 | loss: 0.65729 |  0:00:00s
epoch 21 | loss: 0.65535 |  0:00:01s
epoch 22 | loss: 0.64845 |  0:00:01s
epoch 23 | loss: 0.65655 |  0:00:01s
epoch 24 | loss: 0.65627 |  0:00:01s
epoch 25 | loss: 0.65279 |  0:00:01s
epoch 26 | loss: 0.65325 |  0:00:01s
e

In [38]:
# hyperparameter tuning for TabNet
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
def objective_tabnet(params):
    # Convert categorical feature names to indices
    cat_idxs = [X.columns.get_loc(c) for c in cats]
    cat_dims = [2] * len(cats)  # Assuming binary categorical features

    # Initialize the TabNet model with hyperparameters
    model = SklearnTabNet(
        n_d=int(params['n_d']),
        n_a=int(params['n_a']),
        n_steps=int(params['n_steps']),
        gamma=params['gamma'],
        cat_idxs=cat_idxs,
        cat_dims=cat_dims,
        cat_emb_dim=int(params['cat_emb_dim']),
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=params['lr']),
        scheduler_params=dict(step_size=50, gamma=0.9),
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        verbose=0
    )

    # Perform cross-validation and return the mean AUC
    results = perform_cross_validation(X, y, groups, model, normalize=True, select=[xgb_selector], oversample=True, random_state=42)
    auc_values = [results[i].metrics['AUC'] for i in range(len(results))]
    mean_auc = np.mean(auc_values)

    return {'loss': -mean_auc, 'status': STATUS_OK}
# Define the hyperparameter search space
space_tabnet = {
    'n_d': hp.quniform('n_d', 4, 16, 1),  # Integer between 4 and 16
    'n_a': hp.quniform('n_a', 4, 16, 1),  # Integer between 4 and 16
    'n_steps': hp.quniform('n_steps', 1, 5, 1),  # Integer between 1 and 5
    'gamma': hp.uniform('gamma', 1.0, 2.0),  # Float between 1.0 and 2.0
    'cat_emb_dim': hp.quniform('cat_emb_dim', 1, 8, 1),  # Integer between 1 and 8
    'lr': hp.loguniform('lr', -5, -2)  # Learning rate on a log scale (0.0067 to 0.01)
}
# Run hyperparameter optimization
trials_tabnet = Trials()
best_tabnet = fmin(
    fn=objective_tabnet,
    space=space_tabnet,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials_tabnet
)
print("Best hyperparameters for TabNet:", best_tabnet)
# Update the estimator with the best hyperparameters
estimator = SklearnTabNet(
    n_d=int(best_tabnet['n_d']),
    n_a=int(best_tabnet['n_a']),
    n_steps=int(best_tabnet['n_steps']),
    gamma=best_tabnet['gamma'],
    cat_idxs=[X.columns.get_loc(c) for c in cats],
    cat_dims=[2] * len(cats),  # Assuming binary categorical features
    cat_emb_dim=int(best_tabnet['cat_emb_dim']),
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_tabnet['lr']),
    scheduler_params=dict(step_size=50, gamma=0.9),
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=0
)
# Perform cross-validation with the tuned TabNet model
results = perform_cross_validation(X, y, groups, estimator, normalize=True, select=[xgb_selector], oversample=True, random_state=42)
auc_values = [results[i].metrics['AUC'] for i in range(len(results))]
mean_auc = np.mean(auc_values)
print("Mean AUC after hyperparameter tuning:", mean_auc)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:54:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



epoch 0  | loss: 0.82129 |  0:00:00s                  
epoch 1  | loss: 0.69413 |  0:00:00s                  
epoch 2  | loss: 0.68169 |  0:00:00s                  
epoch 3  | loss: 0.67449 |  0:00:00s                  
epoch 4  | loss: 0.66989 |  0:00:00s                  
epoch 5  | loss: 0.66785 |  0:00:00s                  
epoch 6  | loss: 0.66801 |  0:00:00s                  
epoch 7  | loss: 0.6692  |  0:00:00s                  
epoch 8  | loss: 0.67093 |  0:00:00s                  
epoch 9  | loss: 0.66066 |  0:00:00s                  
epoch 10 | loss: 0.66997 |  0:00:00s                  
epoch 11 | loss: 0.66499 |  0:00:00s                  
epoch 12 | loss: 0.6652  |  0:00:00s                  
epoch 13 | loss: 0.66063 |  0:00:00s                  
epoch 14 | loss: 0.66369 |  0:00:01s                  
epoch 15 | loss: 0.66042 |  0:00:01s                  
epoch 16 | loss: 0.66606 |  0:00:01s                  
epoch 17 | loss: 0.6549  |  0:00:01s                  
epoch 18 |

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:54:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



epoch 1  | loss: 0.66669 |  0:00:00s                  
epoch 2  | loss: 0.66416 |  0:00:00s                  
epoch 3  | loss: 0.65561 |  0:00:00s                  
epoch 4  | loss: 0.65932 |  0:00:00s                  
epoch 5  | loss: 0.66116 |  0:00:00s                  
epoch 6  | loss: 0.65419 |  0:00:00s                  
epoch 7  | loss: 0.65517 |  0:00:00s                  
epoch 8  | loss: 0.66334 |  0:00:00s                  
epoch 9  | loss: 0.65327 |  0:00:00s                  
epoch 10 | loss: 0.65971 |  0:00:00s                  
epoch 11 | loss: 0.659   |  0:00:00s                  
epoch 12 | loss: 0.66051 |  0:00:00s                  
epoch 13 | loss: 0.65711 |  0:00:00s                  
epoch 14 | loss: 0.6592  |  0:00:00s                  
epoch 15 | loss: 0.65609 |  0:00:00s                  
epoch 16 | loss: 0.65753 |  0:00:01s                  
epoch 17 | loss: 0.66103 |  0:00:01s                  
epoch 18 | loss: 0.66029 |  0:00:01s                  
epoch 19 |

...

epoch 1  | loss: 0.70513 |  0:00:00s
epoch 2  | loss: 0.69432 |  0:00:00s
epoch 3  | loss: 0.67477 |  0:00:00s
epoch 4  | loss: 0.67408 |  0:00:00s
epoch 5  | loss: 0.66448 |  0:00:00s
epoch 6  | loss: 0.66114 |  0:00:00s
epoch 7  | loss: 0.67036 |  0:00:00s
epoch 8  | loss: 0.66511 |  0:00:00s
epoch 9  | loss: 0.66068 |  0:00:00s
epoch 10 | loss: 0.66214 |  0:00:00s
epoch 11 | loss: 0.6624  |  0:00:00s
epoch 12 | loss: 0.66165 |  0:00:00s
epoch 13 | loss: 0.66099 |  0:00:00s
epoch 14 | loss: 0.66227 |  0:00:00s
epoch 15 | loss: 0.66577 |  0:00:00s
epoch 16 | loss: 0.6587  |  0:00:00s
epoch 17 | loss: 0.66433 |  0:00:00s
epoch 18 | loss: 0.6621  |  0:00:00s
epoch 19 | loss: 0.6587  |  0:00:01s
epoch 20 | loss: 0.65557 |  0:00:01s
epoch 21 | loss: 0.65922 |  0:00:01s
epoch 22 | loss: 0.66284 |  0:00:01s
epoch 23 | loss: 0.65952 |  0:00:01s
epoch 24 | loss: 0.66066 |  0:00:01s
epoch 25 | loss: 0.65914 |  0:00:01s
epoch 26 | loss: 0.65793 |  0:00:01s
epoch 27 | loss: 0.66025 |  0:00:01s
e

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [17:15:45] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.82813 |  0:00:00s
epoch 1  | loss: 0.67173 |  0:00:00s
epoch 2  | loss: 0.65725 |  0:00:00s
epoch 3  | loss: 0.65813 |  0:00:00s
epoch 4  | loss: 0.65162 |  0:00:00s
epoch 5  | loss: 0.65938 |  0:00:00s
epoch 6  | loss: 0.65699 |  0:00:00s
epoch 7  | loss: 0.65609 |  0:00:00s
epoch 8  | loss: 0.65957 |  0:00:00s
epoch 9  | loss: 0.65455 |  0:00:00s
epoch 10 | loss: 0.65117 |  0:00:00s
epoch 11 | loss: 0.64928 |  0:00:00s
epoch 12 | loss: 0.6484  |  0:00:00s
epoch 13 | loss: 0.65202 |  0:00:00s
epoch 14 | loss: 0.65083 |  0:00:00s
epoch 15 | loss: 0.65236 |  0:00:00s
epoch 16 | loss: 0.65407 |  0:00:00s
epoch 17 | loss: 0.65722 |  0:00:00s
epoch 18 | loss: 0.65453 |  0:00:00s
epoch 19 | loss: 0.65006 |  0:00:00s
epoch 20 | loss: 0.65286 |  0:00:00s
epoch 21 | loss: 0.64635 |  0:00:00s
epoch 22 | loss: 0.65601 |  0:00:00s
epoch 23 | loss: 0.65498 |  0:00:01s
epoch 24 | loss: 0.65045 |  0:00:01s
epoch 25 | loss: 0.64957 |  0:00:01s
epoch 26 | loss: 0.64914 |  0:00:01s
e

/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [17:15:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/dan/miniconda3/envs/intro-machine-learning/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.16024 |  0:00:00s
epoch 1  | loss: 0.71131 |  0:00:00s
epoch 2  | loss: 0.70045 |  0:00:00s
epoch 3  | loss: 0.69166 |  0:00:00s
epoch 4  | loss: 0.67844 |  0:00:00s
epoch 5  | loss: 0.67201 |  0:00:00s
epoch 6  | loss: 0.66742 |  0:00:00s
epoch 7  | loss: 0.65627 |  0:00:00s
epoch 8  | loss: 0.65846 |  0:00:00s
epoch 9  | loss: 0.66    |  0:00:00s
epoch 10 | loss: 0.66268 |  0:00:00s
epoch 11 | loss: 0.65486 |  0:00:00s
epoch 12 | loss: 0.66073 |  0:00:00s
epoch 13 | loss: 0.66184 |  0:00:00s
epoch 14 | loss: 0.66056 |  0:00:00s
epoch 15 | loss: 0.65274 |  0:00:00s
epoch 16 | loss: 0.66015 |  0:00:00s
epoch 17 | loss: 0.66601 |  0:00:00s
epoch 18 | loss: 0.66265 |  0:00:00s
epoch 19 | loss: 0.65902 |  0:00:00s
epoch 20 | loss: 0.65729 |  0:00:00s
epoch 21 | loss: 0.65535 |  0:00:00s
epoch 22 | loss: 0.64845 |  0:00:01s
epoch 23 | loss: 0.65655 |  0:00:01s
epoch 24 | loss: 0.65627 |  0:00:01s
epoch 25 | loss: 0.65279 |  0:00:01s
epoch 26 | loss: 0.65325 |  0:00:01s
e

In [ ]:
# Best hyperparameters for TabNet: {'cat_emb_dim': 4.0, 'gamma': 1.9625566327022135, 'lr': 0.028004677059855897, 'n_a': 12.0, 'n_d': 13.0, 'n_steps': 3.0}
#Mean AUC after hyperparameter tuning: 0.6385634246774857

## Assignment 5. Please try combining all the above methods to push the model performance. (20 pts)

Hint: Methods other than the above methods are also okay to use to improve model performance.

Please avoid data leakage when conducting hyperparameter tuning.


In [ ]:
#######Your code for combing all above mentioned methods to push model performance########